In [2]:
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
from langchain.vectorstores import Qdrant
import os
import pinecone
from tqdm.autonotebook import tqdm

In [8]:

os.environ["COHERE_API_KEY"] = "hv5YTaV6oUo5T9LGOY8F4bBtalGflhTU2FdPtEk3"
# initailize pinecode 

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

In [4]:
# load our document here 
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./BankFAQs.csv", encoding='utf8')

data = loader.load()

In [34]:
# split texts into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1200,
    chunk_overlap  = 200,
    length_function = len,
)

data_chunks = text_splitter.split_documents(data)

In [9]:
# initialize embeddings 
embeddings = CohereEmbeddings(model = "multilingual-22-12")


In [11]:
# vector storage

db = Qdrant.from_documents(data_chunks, embeddings, location=":memory:", collection_name="my_documents", distance_func="Dot")

In [37]:
# index_name = "faqs-dbase"

# docsearch = Pinecone.from_documents(data_chunks, embeddings, index_name=index_name)

In [28]:
# make our prompt 
prompt_template = """Text: {context}

Question: {question}

Answer the question based on the text provided.Change instances where the bank is HDFC Bank to GTbank If the text doesn't contain the answer, reply that the answer is not available."""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [29]:
question = "What do i do when I lost my credit card?"

In [30]:
# question and answer here 
qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0), 
                                 chain_type="stuff", 
                                 retriever=db.as_retriever(), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)
                                 

answer = qa({"query": question})

In [31]:
answer['result']

' If you lose your GTbank credit card, you should call GTbank PhoneBanking immediately to report the loss of your card. This facility is available 24 hrs on all days. The card can also be hotlisted using our Prepaid card NetBanking facility on www.gtbank.com'

In [32]:
def question_and_answer(question):
    qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0), 
                                 chain_type="stuff", 
                                 retriever=db.as_retriever(), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)
                                 

    results = qa({"query": question})

    answer = results['result']

    return answer

In [24]:
question_and_answer("I want to seek for financial advice")

 Answer: You can get Fixed Deposit advice from on the Fixed Deposit summary page on NetBanking. In case your e-mail id is not updated with the bank, you are requested to contact the nearest branch and update your e-mail ID. In case you wish to have a physical advice, you may walk into the nearest HDFC bank branch. A copy of the e-mail advice may be submitted to the branch in case of premature liquidation of the deposit. The e-mail advice copy is as acceptable at our branches as the physical advice.
